https://github.com/aladdinpersson/Machine-Learning-Collection/blob/master/ML/Pytorch/object_detection/YOLO/model.py


In [2]:
## Input
### Import libraries
import torch
import torch.nn as nn
import torch.optim as optim

In [3]:
class MiniBlock(nn.Module):
    def __init__(self,in_channels,num_filters,kernel_size,stride,padding):
        super(MiniBlock, self).__init__()
        self.conv=nn.Conv2d(in_channels=in_channels,out_channels= num_filters,kernel_size=kernel_size,stride=stride,padding=padding,bias =False)
        self.batchnorm=nn.BatchNorm2d(num_filters)
        self.leakyrelu=nn.LeakyReLU(0.1)

    def forward(self,x):
        x=self.conv(x)
        x=self.batchnorm(x)
        x=self.leakyrelu(x)
        return x

In [4]:
architecture_config = [
    (7, 64, 2, 3),
    "M",
    (3, 192, 1, 1),
    "M",
  
    "M",

    "M",

]

In [5]:
class yolov1(nn.Module):
    def __init__(self):
        super(yolov1, self).__init__()
        self.max_pool = nn.MaxPool2d(kernel_size=2,stride=2)  # Max Pooling
        
        self.block1 = nn.Sequential(
        MiniBlock(3,64,7,2,3),
        self.max_pool       
       )
        self.block2 = nn.Sequential(
        MiniBlock(64,192,3,1,1),   
        self.max_pool       
       )
        self.block3 = nn.Sequential(
        MiniBlock(192,128,1,1,0),
        MiniBlock(128,256,3,1,1),
        MiniBlock(256,256,1,1,0),
        MiniBlock(256,512,3,1,1),
        self.max_pool       
       )
       
        self.block4 = nn.Sequential(
        MiniBlock(512,256,1,1,0),
        MiniBlock(256,512,3,1,1),
        MiniBlock(512,256,1,1,0),
        MiniBlock(256,512,3,1,1),
        MiniBlock(512,256,1,1,0),
        MiniBlock(256,512,3,1,1),
        MiniBlock(512,256,1,1,0),
        MiniBlock(256,512,3,1,1),
        
        
        MiniBlock(512,512,1,1,0),
        MiniBlock(512,1024,3,1,1),
        self.max_pool       
       )
        self.block5 = nn.Sequential(
        MiniBlock(1024,512,1,1,0),
        MiniBlock(512,1024,3,1,1),
        MiniBlock(1024,512,1,1,0),
        MiniBlock(512,1024,3,1,1),  
        MiniBlock(1024,1024,3,1,1),  
        MiniBlock(1024,1024,3,2,1),  
        
 
       )
        self.block6 = nn.Sequential(
        MiniBlock(1024,1024,3,1,1),
        MiniBlock(1024,1024,3,1,1),
       )
        self.beforeLastLayer = nn.Linear(7*7*1024, 4096)
        self.lastLayer = nn.Linear(4096, 7*7*30)

    def forward(self, x):
        x = self.block1(x)  # Apply first convolution
        x = self.block2(x)  # Apply average pooling
        x = self.block3(x)  # Apply second convolution
        x = self.block4(x)  # Apply max pooling
        x = self.block5(x)  # Apply max pooling
        x = self.block6(x)  # Apply max pooling
        x = self.beforeLastLayer(torch.flatten(x, 1))
        x = self.lastLayer(x)

        return x

# Create the model instance


In [6]:
image = torch.randn(3,3, 448, 448)
yolo= yolov1()


In [7]:
x=yolo(image)

In [8]:
x.shape

torch.Size([3, 1470])

In [10]:
for i,image in enumerate(x):
    print(i.shape)

AttributeError: 'int' object has no attribute 'shape'